In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import tqdm

import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

class TimeSeriesDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        X = self.data[idx]
        y = self.labels[idx]
        return torch.tensor(X, dtype=torch.float32), torch.tensor(
            y, dtype=torch.float32
        )


def create_time_series_dataloader(X, y, batch_size=32, shuffle=True, num_workers=0):
    dataset = TimeSeriesDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return dataloader

In [ ]:
# get hyperparameters
batch_size = 64
lr1 = 0.005
lr2 = 0.0001
epochs = 100

In [ ]:
# get data
data_train_Hyderabad = np.load(
    "D:/AQI-Forecasting/data_model/dataset_india1/Hyderabad_train.npz",
    allow_pickle=True,
)
train_loader_Hyderabad = create_time_series_dataloader(
    data_train_Hyderabad["samples"],
    data_train_Hyderabad["labels"],
    batch_size=64,
    num_workers=4,
)

# get data
data_train_Dehli = np.load(
    "D:/AQI-Forecasting/data_model/dataset_india1/Dehli_train.npz", allow_pickle=True
)
train_loader_Dehli = create_time_series_dataloader(
    data_train_Dehli["samples"],
    data_train_Dehli["labels"],
    batch_size=64,
    num_workers=4,
)

# get data
data_train_Bengaluru = np.load(
    "D:/AQI-Forecasting/data_model/dataset_india1/Bengaluru_train.npz",
    allow_pickle=True,
)
train_loader_Bengaluru = create_time_series_dataloader(
    data_train_Bengaluru["samples"],
    data_train_Bengaluru["labels"],
    batch_size=64,
    num_workers=4,
)

# get data
data_train_Taiwan = np.load(
    "D:/AQI-Forecasting/data_model/dataset_india1/Taiwan_train.npz", allow_pickle=True
)
train_loader_Taiwan = create_time_series_dataloader(
    data_train_Taiwan["samples"],
    data_train_Taiwan["labels"],
    batch_size=64,
    num_workers=4,
)

In [ ]:
class Cnnlstm(nn.Module):
    def __init__(self):
        super(Cnnlstm, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=8, out_channels=64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(num_features=64)

        self.conv2 = nn.Conv1d(
            in_channels=64, out_channels=64, kernel_size=3, padding=1
        )
        self.bn2 = nn.BatchNorm1d(num_features=64)

        self.conv3 = nn.Conv1d(
            in_channels=64, out_channels=128, kernel_size=5, padding=2
        )
        self.bn3 = nn.BatchNorm1d(num_features=128)

        self.conv4 = nn.Conv1d(
            in_channels=128, out_channels=128, kernel_size=5, padding=2
        )
        self.bn4 = nn.BatchNorm1d(num_features=128)

        self.lstm = nn.LSTM(128, 128, batch_first=True)
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(128, 1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))

        x = x.permute(0, 2, 1)

        x, (hn, cn) = self.lstm(x)
        x = self.global_avg_pool(x.transpose(1, 2))
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [ ]:
# call model
model_Taiwan = Cnnlstm()

model_Dehli = Cnnlstm()

model_Bengaluru = Cnnlstm()

model_Hyderabad = Cnnlstm()

In [ ]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

# define criterion

criterion = nn.MSELoss()


# define optimizer

optimizer_Dehli = optim.SGD(model_Dehli.parameters(), lr=lr1, momentum=0.9)
scheduler_Dehli = lr_scheduler.StepLR(optimizer_Dehli, step_size=50, gamma=0.1)

optimizer_Hyderabad = optim.SGD(model_Hyderabad.parameters(), lr=lr1, momentum=0.9)
scheduler_Hyderabad = lr_scheduler.StepLR(optimizer_Hyderabad, step_size=50, gamma=0.1)

optimizer_Bengaluru = optim.SGD(model_Bengaluru.parameters(), lr=lr1, momentum=0.9)
scheduler_Bengaluru = lr_scheduler.StepLR(optimizer_Bengaluru, step_size=50, gamma=0.1)

optimizer_Taiwan = optim.SGD(model_Taiwan.parameters(), lr=lr1, momentum=0.9)
scheduler_Taiwan = lr_scheduler.StepLR(optimizer_Taiwan, step_size=50, gamma=0.1)

In [ ]:
# get data
data_test_Dehli = np.load(
    "D:/AQI-Forecasting/data_model/dataset_india1/Dehli_test.npz", allow_pickle=True
)
test_loader_Dehli = create_time_series_dataloader(
    data_test_Dehli["samples"], data_test_Dehli["labels"], shuffle=False
)

# get data
data_test_Hyderabad = np.load(
    "D:/AQI-Forecasting/data_model/dataset_india1/Hyderabad_test.npz", allow_pickle=True
)
test_loader_Hyderabad = create_time_series_dataloader(
    data_test_Hyderabad["samples"], data_test_Hyderabad["labels"], shuffle=False
)

# get data
data_test_Bengaluru = np.load(
    "D:/AQI-Forecasting/data_model/dataset_india1/Bengaluru_test.npz", allow_pickle=True
)
test_loader_Bengaluru = create_time_series_dataloader(
    data_test_Bengaluru["samples"], data_test_Bengaluru["labels"], shuffle=False
)

# get data
data_test_Taiwan = np.load(
    "D:/AQI-Forecasting/data_model/dataset_india1/Taiwan_test.npz", allow_pickle=True
)
test_loader_Taiwan = create_time_series_dataloader(
    data_test_Taiwan["samples"], data_test_Taiwan["labels"], shuffle=False
)

In [ ]:
# training
best_loss_Dehli = float("inf")
training_loss_Dehli = []
for epoch in range(epochs):
    running_loss_Dehli = 0.0
    for i_Dehli, data_Dehli in enumerate(
        tqdm.tqdm(
            train_loader_Dehli,
            desc="Training",
            bar_format="{l_bar}{bar:10}{r_bar}{bar:-10b}",
        )
    ):
        # getdata_Dehli
        samples, labels = data_Dehli
        x = samples.permute(0, 2, 1)

        # zero optimizer_Dehli
        optimizer_Dehli.zero_grad()

        # forward pass
        outputs = model_Dehli(x)

        # calculate loss
        loss_Dehli = criterion(outputs, labels)

        # backward pass
        loss_Dehli.backward()
        optimizer_Dehli.step()

        # update running variables
        running_loss_Dehli += loss_Dehli.item()

    running_loss_Dehli /= len(train_loader_Dehli)
    training_loss_Dehli.append(running_loss_Dehli)
    print("Epoch {}/{} | Loss: {:.4f}".format(epoch + 1, epochs, running_loss_Dehli))

    scheduler_Dehli.step()

    # save model_Dehli
    # torch.save(
    #     {
    #         "model_Dehli_state_dict": model_Dehli.state_dict(),
    #         "optimizer_Dehli_state_dict": optimizer_Dehli.state_dict(),
    #         "loss": running_loss_Dehli,
    #     },
    #     f"D:/AQI-Forecasting/train_model_Dehli/model_Dehli_bilstm/logs/checkpoints1/checkpoint_{epoch+1}.ckpt",
    # )
    if best_loss_Dehli > running_loss_Dehli:
        best_loss_Dehli = running_loss_Dehli
        torch.save(
            {
                "model_Dehli_state_dict": model_Dehli.state_dict(),
                "optimizer_Dehli_state_dict": optimizer_Dehli.state_dict(),
                "loss": running_loss_Dehli,
            },
            "D:/AQI-Forecasting/data_model/best_loss_Dehli_checkpoint.ckpt",
        )

In [ ]:
# plot loss
training_loss_Dehli = np.array(training_loss_Dehli)
plt.plot(training_loss_Dehli)
plt.xlabel("Epochs")
plt.ylabel("Loss")

In [ ]:
model_Dehli_test = Cnnlstm()
chetkpoint_Dehli = torch.load(
    "D:/AQI-Forecasting/data_model/best_loss_Dehli_checkpoint.ckpt"
)
model_Dehli_test.load_state_dict(chetkpoint_Dehli["model_Dehli_state_dict"])
model_Dehli_test.eval()

In [ ]:
running_acc_Dehli = 0.0
y_true_Dehli = []
y_pred_Dehli = []
for i_Dehli, test_Data_Dehli in enumerate(
    tqdm.tqdm(
        test_loader_Dehli, desc="Testing", bar_format="{l_bar}{bar:10}{r_bar}{bar:-10b}"
    )
):
    test_sample, test_labels = test_Data_Dehli
    x = test_sample.permute(0, 2, 1)
    # print(x.shape)
    # print(test_sample.shape)
    # print(test_labels.shape)
    outputs1_Dehli = model_Dehli_test(x)
    # print(outputs1_Dehli.shape)

    loss_Dehli = criterion(outputs1_Dehli, test_labels)
    running_loss_Dehli += loss_Dehli.item()

    # Collect true and predicted values
    y_true_Dehli.extend(test_labels.numpy().ravel())
    y_pred_Dehli.extend(outputs1_Dehli.detach().numpy().ravel())
# Calculate average loss_Dehli
running_loss_Dehli /= len(test_loader_Dehli)
print(f"Test loss_Dehli: {running_loss_Dehli:.4f}")

# Calculate additional metrics for regression
y_true_Dehli = np.array(y_true_Dehli)
y_pred_Dehli = np.array(y_pred_Dehli)
print(y_true_Dehli.shape)
print(y_pred_Dehli.shape)
print(y_true_Dehli[0])
mse_Dehli = mean_squared_error(y_true_Dehli, y_pred_Dehli)
mae_Dehli = mean_absolute_error(y_true_Dehli, y_pred_Dehli)
r2_Dehli = r2_score(y_true_Dehli, y_pred_Dehli)

print(f"Mean Squared Error: {mse_Dehli:.4f}")
print(f"Mean Absolute Error: {mae_Dehli:.4f}")
print(f"R^2 Score: {r2_Dehli:.4f}")

In [ ]:
y_true_Dehli = np.array(y_true_Dehli)
y_pred_Dehli = np.array(y_pred_Dehli)

plt.figure(figsize=(10, 6))

# Vẽ đường biểu diễn giá trị thực tế
plt.plot(y_true_Dehli, color="blue", label="Actual AQI")

# Vẽ đường biểu diễn giá trị dự đoán
plt.plot(y_pred_Dehli, color="green", label="Predicted AQI")

# Đặt tiêu đề và nhãn cho đồ thị
plt.title("AQI Prediction (Multivariate)")
plt.xlabel("Samples")
plt.ylabel("AQI")

# Hiển thị chú thích
plt.legend()

# Hiển thị đồ thị
plt.show()

In [ ]:
# training
best_loss_Bengaluru = float("inf")
training_loss_Bengaluru = []
for epoch in range(epochs):
    running_loss_Bengaluru = 0.0
    for i_Bengaluru, data_Bengaluru in enumerate(
        tqdm.tqdm(
            train_loader_Bengaluru,
            desc="Training",
            bar_format="{l_bar}{bar:10}{r_bar}{bar:-10b}",
        )
    ):
        # getdata_Bengaluru
        samples, labels = data_Bengaluru
        
        x = samples.permute(0, 2, 1)

        # zero optimizer_Bengaluru
        optimizer_Bengaluru.zero_grad()

        # forward pass
        outputs = model_Bengaluru(x)

        # calculate loss
        loss_Bengaluru = criterion(outputs, labels)

        # backward pass
        loss_Bengaluru.backward()
        optimizer_Bengaluru.step()

        # update running variables
        running_loss_Bengaluru += loss_Bengaluru.item()

    running_loss_Bengaluru /= len(train_loader_Bengaluru)
    training_loss_Bengaluru.append(running_loss_Bengaluru)
    print(
        "Epoch {}/{} | Loss: {:.4f}".format(epoch + 1, epochs, running_loss_Bengaluru)
    )

    scheduler_Bengaluru.step()

    # save model_Bengaluru
    # torch.save(
    #     {
    #         "model_Bengaluru_state_dict": model_Bengaluru.state_dict(),
    #         "optimizer_Bengaluru_state_dict": optimizer_Bengaluru.state_dict(),
    #         "loss": running_loss_Bengaluru,
    #     },
    #     f"D:/AQI-Forecasting/train_model_Bengaluru/model_Bengaluru_bilstm/logs/checkpoints1/checkpoint_{epoch+1}.ckpt",
    # )
    if best_loss_Bengaluru > running_loss_Bengaluru:
        best_loss_Bengaluru = running_loss_Bengaluru
        torch.save(
            {
                "model_Bengaluru_state_dict": model_Bengaluru.state_dict(),
                "optimizer_Bengaluru_state_dict": optimizer_Bengaluru.state_dict(),
                "loss": running_loss_Bengaluru,
            },
            "D:/AQI-Forecasting/data_model/best_loss_Bengaluru_checkpoint.ckpt",
        )

In [ ]:
# plot loss
training_loss_Bengaluru = np.array(training_loss_Bengaluru)
plt.plot(training_loss_Bengaluru)
plt.xlabel("Epochs")
plt.ylabel("Loss")

In [ ]:
model_Bengaluru_test = Cnnlstm()
chetkpoint_Bengaluru = torch.load("D:/AQI-Forecasting/data_model/best_loss_Bengaluru_checkpoint.ckpt")
model_Bengaluru_test.load_state_dict(chetkpoint_Bengaluru["model_Bengaluru_state_dict"])
model_Bengaluru_test.eval()

In [ ]:
running_acc_Bengaluru = 0.0
y_true_Bengaluru = []
y_pred_Bengaluru = []
for i_Bengaluru, test_Data_Bengaluru in enumerate(
    tqdm.tqdm(
        test_loader_Bengaluru,
        desc="Testing",
        bar_format="{l_bar}{bar:10}{r_bar}{bar:-10b}",
    )
):
    test_sample1, test_labels1 = test_Data_Bengaluru
    x = test_sample1.permute(0, 2, 1)
    # print(x.shape)
    # print(test_sample.shape)
    # print(test_labels.shape)
    outputs1_Bengaluru = model_Bengaluru_test(x)
    # print(outputs1_Bengaluru.shape)

    loss_Bengaluru = criterion(outputs1_Bengaluru, test_labels1)
    running_loss_Bengaluru += loss_Bengaluru.item()

    # Collect true and predicted values
    y_true_Bengaluru.extend(test_labels1.numpy().ravel())
    y_pred_Bengaluru.extend(outputs1_Bengaluru.detach().numpy().ravel())
# Calculate average loss_Bengaluru
running_loss_Bengaluru /= len(test_loader_Bengaluru)
print(f"Test loss_Bengaluru: {running_loss_Bengaluru:.4f}")

# Calculate additional metrics for regression
y_true_Bengaluru = np.array(y_true_Bengaluru)
y_pred_Bengaluru = np.array(y_pred_Bengaluru)
print(y_true_Bengaluru.shape)
print(y_pred_Bengaluru.shape)
print(y_true_Bengaluru[0])
mse_Bengaluru = mean_squared_error(y_true_Bengaluru, y_pred_Bengaluru)
mae_Bengaluru = mean_absolute_error(y_true_Bengaluru, y_pred_Bengaluru)
r2_Bengaluru = r2_score(y_true_Bengaluru, y_pred_Bengaluru)

print(f"Mean Squared Error: {mse_Bengaluru:.4f}")
print(f"Mean Absolute Error: {mae_Bengaluru:.4f}")
print(f"R^2 Score: {r2_Bengaluru:.4f}")

In [ ]:
y_true_Bengaluru = np.array(y_true_Bengaluru)
y_pred_Bengaluru = np.array(y_pred_Bengaluru)

plt.figure(figsize=(10, 6))

# Vẽ đường biểu diễn giá trị thực tế
plt.plot(y_true_Bengaluru, color="blue", label="Actual AQI")

# Vẽ đường biểu diễn giá trị dự đoán
plt.plot(y_pred_Bengaluru, color="green", label="Predicted AQI")

# Đặt tiêu đề và nhãn cho đồ thị
plt.title("AQI Prediction (Multivariate)")
plt.xlabel("Samples")
plt.ylabel("AQI")

# Hiển thị chú thích
plt.legend()

# Hiển thị đồ thị
plt.show()

In [ ]:
# training
best_loss_Hyderabad = float("inf")
training_loss_Hyderabad = []
for epoch in range(epochs):
    running_loss_Hyderabad = 0.0
    for i_Hyderabad, data_Hyderabad in enumerate(
        tqdm.tqdm(
            train_loader_Hyderabad,
            desc="Training",
            bar_format="{l_bar}{bar:10}{r_bar}{bar:-10b}",
        )
    ):
        # getdata_Hyderabad
        samples, labels = data_Hyderabad
        
        x = samples.permute(0, 2, 1)

        # zero optimizer_Hyderabad
        optimizer_Hyderabad.zero_grad()

        # forward pass
        outputs = model_Hyderabad(x)

        # calculate loss
        loss_Hyderabad = criterion(outputs, labels)

        # backward pass
        loss_Hyderabad.backward()
        optimizer_Hyderabad.step()

        # update running variables
        running_loss_Hyderabad += loss_Hyderabad.item()

    running_loss_Hyderabad /= len(train_loader_Hyderabad)
    training_loss_Hyderabad.append(running_loss_Hyderabad)
    print(
        "Epoch {}/{} | Loss: {:.4f}".format(epoch + 1, epochs, running_loss_Hyderabad)
    )

    scheduler_Hyderabad.step()

    # save model_Hyderabad
    # torch.save(
    #     {
    #         "model_Hyderabad_state_dict": model_Hyderabad.state_dict(),
    #         "optimizer_Hyderabad_state_dict": optimizer_Hyderabad.state_dict(),
    #         "loss": running_loss_Hyderabad,
    #     },
    #     f"D:/AQI-Forecasting/train_model_Hyderabad/model_Hyderabad_bilstm/logs/checkpoints1/checkpoint_{epoch+1}.ckpt",
    # )
    if best_loss_Hyderabad > running_loss_Hyderabad:
        best_loss_Hyderabad = running_loss_Hyderabad
        torch.save(
            {
                "model_Hyderabad_state_dict": model_Hyderabad.state_dict(),
                "optimizer_Hyderabad_state_dict": optimizer_Hyderabad.state_dict(),
                "loss": running_loss_Hyderabad,
            },
            "D:/AQI-Forecasting/data_model/best_loss_Hyderabad_checkpoint.ckpt",
        )

In [ ]:
# plot loss
training_loss_Hyderabad = np.array(training_loss_Hyderabad)
plt.plot(training_loss_Hyderabad)
plt.xlabel("Epochs")
plt.ylabel("Loss")

In [ ]:
model_Hyderabad_test = Cnnlstm()
chetkpoint_Hyderabad = torch.load("D:/AQI-Forecasting/data_model/best_loss_Hyderabad_checkpoint.ckpt")
model_Hyderabad_test.load_state_dict(chetkpoint_Hyderabad["model_Hyderabad_state_dict"])
model_Hyderabad_test.eval()

In [ ]:
running_acc_Hyderabad = 0.0
y_true_Hyderabad = []
y_pred_Hyderabad = []
for i_Hyderabad, test_data_Hyderabad in enumerate(
    tqdm.tqdm(
        test_loader_Hyderabad,
        desc="Testing",
        bar_format="{l_bar}{bar:10}{r_bar}{bar:-10b}",
    )
):
    test_sample2, test_labels2 = test_data_Hyderabad
    x = test_sample2.permute(0, 2, 1)
    # print(x.shape)
    # print(test_sample.shape)
    # print(test_labels.shape)
    outputs1_Hyderabad = model_Hyderabad_test(x)
    # print(outputs1_Hyderabad.shape)

    loss_Hyderabad = criterion(outputs1_Hyderabad, test_labels2)
    running_loss_Hyderabad += loss_Hyderabad.item()

    # Collect true and predicted values
    y_true_Hyderabad.extend(test_labels2.numpy().ravel())
    y_pred_Hyderabad.extend(outputs1_Hyderabad.detach().numpy().ravel())
# Calculate average loss_Hyderabad
running_loss_Hyderabad /= len(test_loader_Hyderabad)
print(f"Test loss_Hyderabad: {running_loss_Hyderabad:.4f}")

# Calculate additional metrics for regression
y_true_Hyderabad = np.array(y_true_Hyderabad)
y_pred_Hyderabad = np.array(y_pred_Hyderabad)
print(y_true_Hyderabad.shape)
print(y_pred_Hyderabad.shape)
print(y_true_Hyderabad[0])
mse_Hyderabad = mean_squared_error(y_true_Hyderabad, y_pred_Hyderabad)
mae_Hyderabad = mean_absolute_error(y_true_Hyderabad, y_pred_Hyderabad)
r2_Hyderabad = r2_score(y_true_Hyderabad, y_pred_Hyderabad)

print(f"Mean Squared Error: {mse_Hyderabad:.4f}")
print(f"Mean Absolute Error: {mae_Hyderabad:.4f}")
print(f"R^2 Score: {r2_Hyderabad:.4f}")

In [ ]:
y_true_Hyderabad = np.array(y_true_Hyderabad)
y_pred_Hyderabad = np.array(y_pred_Hyderabad)

plt.figure(figsize=(10, 6))

# Vẽ đường biểu diễn giá trị thực tế
plt.plot(y_true_Hyderabad, color="blue", label="Actual AQI")

# Vẽ đường biểu diễn giá trị dự đoán
plt.plot(y_pred_Hyderabad, color="green", label="Predicted AQI")

# Đặt tiêu đề và nhãn cho đồ thị
plt.title("AQI Prediction (Multivariate)")
plt.xlabel("Samples")
plt.ylabel("AQI")

# Hiển thị chú thích
plt.legend()

# Hiển thị đồ thị
plt.show()

In [ ]:
# training
best_loss_Taiwan = float("inf")
training_loss_Taiwan = []
for epoch in range(epochs):
    running_loss_Taiwan = 0.0
    for i_Taiwan, data_Taiwan in enumerate(
        tqdm.tqdm(
            train_loader_Taiwan,
            desc="Training",
            bar_format="{l_bar}{bar:10}{r_bar}{bar:-10b}",
        )
    ):
        # getdata_Taiwan
        samples, labels = data_Taiwan
        
        x = samples.permute(0, 2, 1)

        # zero optimizer_Taiwan
        optimizer_Taiwan.zero_grad()

        # forward pass
        outputs = model_Taiwan(x)

        # calculate loss
        loss_Taiwan = criterion(outputs, labels)

        # backward pass
        loss_Taiwan.backward()
        optimizer_Taiwan.step()

        # update running variables
        running_loss_Taiwan += loss_Taiwan.item()

    running_loss_Taiwan /= len(train_loader_Taiwan)
    training_loss_Taiwan.append(running_loss_Taiwan)
    print("Epoch {}/{} | Loss: {:.4f}".format(epoch + 1, epochs, running_loss_Taiwan))

    scheduler_Taiwan.step()

    # save model_Taiwan
    # torch.save(
    #     {
    #         "model_Taiwan_state_dict": model_Taiwan.state_dict(),
    #         "optimizer_Taiwan_state_dict": optimizer_Taiwan.state_dict(),
    #         "loss": running_loss_Taiwan,
    #     },
    #     f"D:/AQI-Forecasting/train_model_Taiwan/model_Taiwan_bilstm/logs/checkpoints1/checkpoint_{epoch+1}.ckpt",
    # )
    if best_loss_Taiwan > running_loss_Taiwan:
        best_loss_Taiwan = running_loss_Taiwan
        torch.save(
            {
                "model_Taiwan_state_dict": model_Taiwan.state_dict(),
                "optimizer_Taiwan_state_dict": optimizer_Taiwan.state_dict(),
                "loss": running_loss_Taiwan,
            },
            "D:/AQI-Forecasting/data_model/best_loss_Taiwan_checkpoint.ckpt",
        )

In [ ]:
# plot loss
training_loss_Taiwan = np.array(training_loss_Taiwan)
plt.plot(training_loss_Taiwan)
plt.xlabel("Epochs")
plt.ylabel("Loss")

In [ ]:
model_Taiwan_test = Cnnlstm()
chetkpoint_Taiwan = torch.load("D:/AQI-Forecasting/data_model/best_loss_Taiwan_checkpoint.ckpt")
model_Taiwan_test.load_state_dict(chetkpoint_Taiwan["model_Taiwan_state_dict"])
model_Taiwan_test.eval()

In [ ]:
running_acc_Taiwan = 0.0
y_true_Taiwan = []
y_pred_Taiwan = []
for i_Taiwan, test_data_Taiwan in enumerate(
    tqdm.tqdm(
        test_loader_Taiwan,
        desc="Testing",
        bar_format="{l_bar}{bar:10}{r_bar}{bar:-10b}",
    )
):
    test_sample3, test_labels3 = test_data_Taiwan
    x = test_sample3.rmute(0, 2, 1)
    # print(x.shape)
    # print(test_sample.shape)
    # print(test_labels.shape)
    outputs1_Taiwan = model_Taiwan_test(x)
    # print(outputs1_Taiwan.shape)

    loss_Taiwan = criterion(outputs1_Taiwan, test_labels3)
    running_loss_Taiwan += loss_Taiwan.item()

    # Collect true and predicted values
    y_true_Taiwan.extend(test_labels3.numpy().ravel())
    y_pred_Taiwan.extend(outputs1_Taiwan.detach().numpy().ravel())
# Calculate average loss_Taiwan
running_loss_Taiwan /= len(test_loader_Taiwan)
print(f"Test loss_Taiwan: {running_loss_Taiwan:.4f}")

# Calculate additional metrics for regression
y_true_Taiwan = np.array(y_true_Taiwan)
y_pred_Taiwan = np.array(y_pred_Taiwan)
print(y_true_Taiwan.shape)
print(y_pred_Taiwan.shape)
print(y_true_Taiwan[0])
mse_Taiwan = mean_squared_error(y_true_Taiwan, y_pred_Taiwan)
mae_Taiwan = mean_absolute_error(y_true_Taiwan, y_pred_Taiwan)
r2_Taiwan = r2_score(y_true_Taiwan, y_pred_Taiwan)

print(f"Mean Squared Error: {mse_Taiwan:.4f}")
print(f"Mean Absolute Error: {mae_Taiwan:.4f}")
print(f"R^2 Score: {r2_Taiwan:.4f}")

In [ ]:
y_true_Taiwan = np.array(y_true_Taiwan)
y_pred_Taiwan = np.array(y_pred_Taiwan)

plt.figure(figsize=(10, 6))

# Vẽ đường biểu diễn giá trị thực tế
plt.plot(y_true_Taiwan, color="blue", label="Actual AQI")

# Vẽ đường biểu diễn giá trị dự đoán
plt.plot(y_pred_Taiwan, color="green", label="Predicted AQI")

# Đặt tiêu đề và nhãn cho đồ thị
plt.title("AQI Prediction (Multivariate)")
plt.xlabel("Samples")
plt.ylabel("AQI")

# Hiển thị chú thích
plt.legend()

# Hiển thị đồ thị
plt.show()